In [1]:
##imports
import time
import numpy as np
import sys
import random
import os
import warnings
import datetime
import argparse

import torch
from torch.utils.tensorboard import SummaryWriter
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.utils.data.distributed
from easydict import EasyDict as edict
from tqdm import tqdm

sys.path.append('./')

from data_process.ttnet_dataloader import create_train_val_dataloader, create_test_dataloader
from models.model_utils import create_model, load_pretrained_model, make_data_parallel, resume_model, get_num_parameters
from models.model_utils import freeze_model
from utils.train_utils import create_optimizer, create_lr_scheduler, get_saved_state, save_checkpoint
from utils.train_utils import reduce_tensor, to_python_float
from utils.misc import AverageMeter, ProgressMeter
from utils.logger import Logger
#from config.config import parse_configs
from utils.misc import make_folder

###### Config

In [2]:
def parse_configs():
    parser = argparse.ArgumentParser(description='TTNet Implementation')
    parser.add_argument('--seed', type=int, default=2020, help='re-produce the results with seed random')
    parser.add_argument('--saved_fn', type=str, default='ttnet', metavar='FN', help='The name using for saving logs, models,...')
    ####################################################################
    ##############     Model configs            ###################
    ####################################################################
    parser.add_argument('-a', '--arch', type=str, default='ttnet', metavar='ARCH', help='The name of the model architecture')
    parser.add_argument('--dropout_p', type=float, default=0.5, metavar='P', help='The dropout probability of the model')
    parser.add_argument('--multitask_learning', action='store_true', help='If true, the weights of different losses will be learnt (train).'
                             'If false, a regular sum of different losses will be applied')
    parser.add_argument('--no_local', action='store_true', help='If true, no local stage for ball detection.')
    parser.add_argument('--no_event', action='store_true', help='If true, no event spotting detection.')
    parser.add_argument('--no_seg', action='store_true', help='If true, no segmentation module.')
    parser.add_argument('--pretrained_path', type=str, default=None, metavar='PATH', help='the path of the pretrained checkpoint')
    parser.add_argument('--overwrite_global_2_local', action='store_true', help='If true, the weights of the local stage will be overwritten by the global stage.')

    ####################################################################
    ##############     Dataloader and Running configs            #######
    ####################################################################
    parser.add_argument('--working-dir', type=str, default='../../', metavar='PATH', help='the ROOT working directory')
    parser.add_argument('--no-val', action='store_true', help='If true, use all data for training, no validation set')
    parser.add_argument('--no-test', action='store_true', help='If true, dont evaluate the model on the test set')
    parser.add_argument('--val-size', type=float, default=0.2, help='The size of validation set')
    parser.add_argument('--smooth-labelling', action='store_true', help='If true, smoothly make the labels of event spotting')
    parser.add_argument('--num_samples', type=int, default=None, help='Take a subset of the dataset to run and debug')
    parser.add_argument('--num_workers', type=int, default=4, help='Number of threads for loading data')
    parser.add_argument('--batch_size', type=int, default=8, help='mini-batch size (default: 8), this is the total'
                             'batch size of all GPUs on the current node when using'
                             'Data Parallel or Distributed Data Parallel')
    parser.add_argument('--print_freq', type=int, default=50, metavar='N', help='print frequency (default: 50)')
    parser.add_argument('--checkpoint_freq', type=int, default=2, metavar='N', help='frequency of saving checkpoints (default: 2)')
    parser.add_argument('--sigma', type=float, default=1., metavar='SIGMA', help='standard deviation of the 1D Gaussian for the ball position target')
    parser.add_argument('--thresh_ball_pos_mask', type=float, default=0.05, metavar='THRESH', help='the lower thresh for the 1D Gaussian of the ball position target')
    ####################################################################
    ##############     Training strategy            ###################
    ####################################################################

    parser.add_argument('--start_epoch', type=int, default=1, metavar='N', help='the starting epoch')
    parser.add_argument('--num_epochs', type=int, default=30, metavar='N', help='number of total epochs to run')
    parser.add_argument('--lr', type=float, default=1e-3, metavar='LR', help='initial learning rate')
    parser.add_argument('--minimum_lr', type=float, default=1e-7, metavar='MIN_LR', help='minimum learning rate during training')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='momentum')
    parser.add_argument('-wd', '--weight_decay', type=float, default=0., metavar='WD', help='weight decay (default: 1e-6)')
    parser.add_argument('--optimizer_type', type=str, default='adam', metavar='OPTIMIZER', help='the type of optimizer, it can be sgd or adam')
    parser.add_argument('--lr_type', type=str, default='plateau', metavar='SCHEDULER', help='the type of the learning rate scheduler (steplr or ReduceonPlateau)')
    parser.add_argument('--lr_factor', type=float, default=0.5, metavar='FACTOR', help='reduce the learning rate with this factor')
    parser.add_argument('--lr_step_size', type=int, default=5, metavar='STEP_SIZE', help='step_size of the learning rate when using steplr scheduler')
    parser.add_argument('--lr_patience', type=int, default=3, metavar='N', help='patience of the learning rate when using ReduceoPlateau scheduler')
    parser.add_argument('--earlystop_patience', type=int, default=None, metavar='N', help='Early stopping the training process if performance is not improved within this value')
    parser.add_argument('--freeze_global', action='store_true', help='If true, no update/train weights for the global stage of ball detection.')
    parser.add_argument('--freeze_local', action='store_true', help='If true, no update/train weights for the local stage of ball detection.')
    parser.add_argument('--freeze_event', action='store_true', help='If true, no update/train weights for the event module.')
    parser.add_argument('--freeze_seg', action='store_true', help='If true, no update/train weights for the segmentation module.')

    ####################################################################
    ##############     Loss weight            ###################
    ####################################################################
    parser.add_argument('--bce_weight', type=float, default=0.5, help='The weight of BCE loss in segmentation module, the dice_loss weight = 1- bce_weight')
    parser.add_argument('--global_weight', type=float, default=1., help='The weight of loss of the global stage for ball detection')
    parser.add_argument('--local_weight', type=float, default=1., help='The weight of loss of the local stage for ball detection')
    parser.add_argument('--event_weight', type=float, default=1., help='The weight of loss of the event spotting module')
    parser.add_argument('--seg_weight', type=float, default=1., help='The weight of BCE loss in segmentation module')

    ####################################################################
    ##############     Distributed Data Parallel            ############
    ####################################################################
    parser.add_argument('--world-size', default=-1, type=int, metavar='N', help='number of nodes for distributed training')
    parser.add_argument('--rank', default=-1, type=int, metavar='N', help='node rank for distributed training')
    parser.add_argument('--dist-url', default='tcp://127.0.0.1:29500', type=str, help='url used to set up distributed training')
    parser.add_argument('--dist-backend', default='nccl', type=str, help='distributed backend')
    parser.add_argument('--gpu_idx', default=0, type=int, help='GPU index to use.')
    parser.add_argument('--no_cuda', action='store_true', help='If true, cuda is not used.')
    parser.add_argument('--multiprocessing-distributed', action='store_true', help='Use multi-processing distributed training to launch '
                             'N processes per node, which has N GPUs. This is the '
                             'fastest way to use PyTorch for either single node or '
                             'multi node data parallel training')
    ####################################################################
    ##############     Evaluation configurations     ###################
    ####################################################################
    parser.add_argument('--evaluate', action='store_true', help='only evaluate the model, not training')
    parser.add_argument('--resume_path', type=str, default=None, metavar='PATH', help='the path of the resumed checkpoint')
    parser.add_argument('--use_best_checkpoint', action='store_true', help='If true, choose the best model on val set, otherwise choose the last model')
    parser.add_argument('--seg_thresh', type=float, default=0.5, help='threshold of the segmentation output')
    parser.add_argument('--event_thresh', type=float, default=0.5, help='threshold of the event spotting output')
    parser.add_argument('--save_test_output', action='store_true', help='If true, the image of testing phase will be saved')

    ####################################################################
    ##############     Demonstration configurations     ###################
    ####################################################################
    parser.add_argument('--video_path', type=str, default=None, metavar='PATH', help='the path of the video that needs to demo')
    parser.add_argument('--output_format', type=str, default='text', metavar='PATH', help='the type of the demo output')
    parser.add_argument('--show_image', action='store_true', help='If true, show the image during demostration')
    parser.add_argument('--save_demo_output', action='store_true', help='If true, the image of demonstration phase will be saved')

    configs = edict(vars(parser.parse_args([])))

    return configs

In [3]:
def config_process(configs):
    ####################################################################
    ############## Hardware configurations ############################
    ####################################################################
    configs.device = torch.device('cpu' if configs.no_cuda else 'cuda')
    configs.ngpus_per_node = torch.cuda.device_count()

    configs.pin_memory = True

    ####################################################################
    ##############     Data configs            ###################
    ####################################################################
    configs.dataset_dir = os.path.join(configs.working_dir, 'dataset')
    configs.train_game_list = ['game_1']#, 'game_2', 'game_3', 'game_4', 'game_5']
    configs.test_game_list = ['test_1']#, 'test_2', 'test_3', 'test_4', 'test_5', 'test_6', 'test_7']
    configs.events_dict = {
        'bounce': 0,
        'net': 1,
        'empty_event': 2
    }
    configs.events_weights_loss_dict = {
        'bounce': 1.,
        'net': 3.,
    }
    configs.events_weights_loss = (configs.events_weights_loss_dict['bounce'], configs.events_weights_loss_dict['net'])
    configs.num_events = len(configs.events_weights_loss_dict)  # Just "bounce" and "net hits"
    configs.num_frames_sequence = 9

    configs.org_size = (1920, 1080)
    configs.input_size = (320, 128)

    configs.tasks = ['global', 'local', 'event', 'seg']
    if configs.no_local:
        if 'local' in configs.tasks:
            configs.tasks.remove('local')
        if 'event' in configs.tasks:
            configs.tasks.remove('event')
    if configs.no_event:
        if 'event' in configs.tasks:
            configs.tasks.remove('event')
    if configs.no_seg:
        if 'seg' in configs.tasks:
            configs.tasks.remove('seg')

    # Compose loss weight for tasks, normalize the weights later
    loss_weight_dict = {
        'global': configs.global_weight,
        'local': configs.local_weight,
        'event': configs.event_weight,
        'seg': configs.seg_weight
    }
    configs.tasks_loss_weight = [loss_weight_dict[task] for task in configs.tasks]

    configs.freeze_modules_list = []
    if configs.freeze_global:
        configs.freeze_modules_list.append('ball_global_stage')
    if configs.freeze_local:
        configs.freeze_modules_list.append('ball_local_stage')
    if configs.freeze_event:
        configs.freeze_modules_list.append('events_spotting')
    if configs.freeze_seg:
        configs.freeze_modules_list.append('segmentation')

    ####################################################################
    ############## logs, Checkpoints, and results dir ########################
    ####################################################################
    configs.checkpoints_dir = os.path.join(configs.working_dir, 'checkpoints', configs.saved_fn)
    configs.logs_dir = os.path.join(configs.working_dir, 'logs', configs.saved_fn)
    configs.use_best_checkpoint = True

    if configs.use_best_checkpoint:
        configs.saved_weight_name = os.path.join(configs.checkpoints_dir, '{}_best.pth'.format(configs.saved_fn))
    else:
        configs.saved_weight_name = os.path.join(configs.checkpoints_dir, '{}.pth'.format(configs.saved_fn))

    configs.results_dir = os.path.join(configs.working_dir, 'results')

    make_folder(configs.checkpoints_dir)
    make_folder(configs.logs_dir)
    make_folder(configs.results_dir)

    if configs.save_test_output:
        configs.saved_dir = os.path.join(configs.results_dir, configs.saved_fn)
        make_folder(configs.saved_dir)

    if configs.save_demo_output:
        configs.save_demo_dir = os.path.join(configs.results_dir, 'demo', configs.saved_fn)
        make_folder(configs.save_demo_dir)

    return configs

In [5]:
PHASE = 3

In [6]:
configs = parse_configs()

In [7]:
if PHASE == 1: 
    #Phase 1
    configs.working_dir      = "../"
    configs.saved_fn         = 'ttnet_1st_phase_opentt'
    configs.no_val           = True
    configs.batch_size       = 24
    configs.num_workers      = 8
    configs.lr               = 0.001 
    configs.lr_type          = 'step_lr' 
    configs.lr_step_size     = 10 
    configs.lr_factor        = 0.1
    configs.gpu_idx          = 0 
    configs.global_weight    = 5. 
    configs.seg_weight       = 1.
    configs.no_event         = True
    configs.no_local         = True
    configs.print_freq       = 500
    configs.smooth_labelling = True

if PHASE == 2:
    #Phase 2
    configs.working_dir                = "../"
    configs.batch_size                 = 24
    configs.num_workers                = 8
    configs.saved_fn                   = 'ttnet_2nd_phase_opentt' 
    configs.no_val                     = True  
    configs.lr                         = 0.001 
    configs.lr_type                    = 'step_lr' 
    configs.lr_step_size               =  10 
    configs.lr_factor                  = 0.1 
    configs.gpu_idx                    = 0 
    configs.global_weight              = 0. 
    configs.event_weight               = 2. 
    configs.local_weight               = 1. 
    configs.pretrained_path            = "../checkpoints/ttnet_1st_phase_opentt/ttnet_1st_phase_opentt_epoch_30.pth"
    configs.overwrite_global_2_local   = True
    configs.freeze_global              = True
    configs.smooth_labelling           = True
    configs.freeze_seg                 = True
    configs.sigma                      = 1.0
    configs.print_freq                 = 500   
    
if PHASE == 3:
    #Phase 3
    configs.working_dir       = "../"
    configs.saved_fn          = 'ttnet_3nd_phase' 
    configs.batch_size        = 24
    configs.num_workers       = 8
    
    configs.no_val            = True  
    configs.lr                = 0.0001 
    configs.lr_type           = 'step_lr' 
    configs.lr_step_size      =  10 
    configs.lr_factor         = 0.2 
    configs.gpu_idx           = 0 
    configs.global_weight     = 1. 
    configs.event_weight      = 1. 
    configs.local_weight      = 1. 
    configs.pretrained_path   = "../checkpoints/ttnet_2nd_phase_opentt/ttnet_2nd_phase_opentt_epoch_30.pth"
    configs.smooth_labelling  = True
#     configs.sigma             = 1.0
    configs.print_freq        = 500

In [8]:
configs = config_process(configs)

In [9]:
configs.freeze_modules_list

[]

In [10]:
if configs.seed is not None:
    random.seed(configs.seed)
    np.random.seed(configs.seed)
    torch.manual_seed(configs.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
if configs.gpu_idx is not None:
    warnings.warn('You have chosen a specific GPU. This will completely '
                      'disable data parallelism.')

/home/stupa/miniconda3/envs/ttnet/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  


In [11]:
configs.distributed = configs.world_size > 1 or configs.multiprocessing_distributed

In [12]:
if configs.gpu_idx is not None:
    print("Use GPU: {} for training".format(configs.gpu_idx))
    configs.device = torch.device('cuda:{}'.format(configs.gpu_idx))

Use GPU: 0 for training


In [13]:
configs.is_master_node = (not configs.distributed) or (configs.distributed and (configs.rank % configs.ngpus_per_node == 0))

In [14]:
configs.is_master_node

True

In [15]:
if configs.is_master_node:
    logger = Logger(configs.logs_dir, configs.saved_fn)
    logger.info('>>> Created a new logger')
    logger.info('>>> configs: {}'.format(configs))
    tb_writer = SummaryWriter(log_dir=os.path.join(configs.logs_dir, 'tensorboard'))
else:
    logger = None
    tb_writer = None

2022-12-19 20:56:19,882: logger.py - info(), at Line 38:INFO:
>>> Created a new logger
2022-12-19 20:56:19,883: logger.py - info(), at Line 38:INFO:
>>> configs: {'seed': 2020, 'saved_fn': 'ttnet_3nd_phase', 'arch': 'ttnet', 'dropout_p': 0.5, 'multitask_learning': False, 'no_local': False, 'no_event': False, 'no_seg': False, 'pretrained_path': '../checkpoints/ttnet_2nd_phase_opentt/ttnet_2nd_phase_opentt_epoch_30.pth', 'overwrite_global_2_local': False, 'working_dir': '../', 'no_val': True, 'no_test': False, 'val_size': 0.2, 'smooth_labelling': True, 'num_samples': None, 'num_workers': 8, 'batch_size': 24, 'print_freq': 500, 'checkpoint_freq': 2, 'sigma': 1.0, 'thresh_ball_pos_mask': 0.05, 'start_epoch': 1, 'num_epochs': 30, 'lr': 0.0001, 'minimum_lr': 1e-07, 'momentum': 0.9, 'weight_decay': 0.0, 'optimizer_type': 'adam', 'lr_type': 'step_lr', 'lr_factor': 0.2, 'lr_step_size': 10, 'lr_patience': 3, 'earlystop_patience': None, 'freeze_global': False, 'freeze_local': False, 'freeze_event

##### Model creation

In [16]:
model = create_model(configs)

In [17]:
model = make_data_parallel(model, configs)

# Freeze model
model = freeze_model(model, configs.freeze_modules_list)

#### Train supports

In [18]:
if configs.is_master_node:
    num_parameters = get_num_parameters(model)
    logger.info('number of trained parameters of the model: {}'.format(num_parameters))

optimizer = create_optimizer(configs, model)
lr_scheduler = create_lr_scheduler(optimizer, configs)
best_val_loss = np.inf
earlystop_count = 0
is_best = False

2022-12-19 20:56:21,892: logger.py - info(), at Line 38:INFO:
number of trained parameters of the model: 16091813


In [19]:
configs.is_master_node

True

In [20]:
if configs.pretrained_path is not None:
    model = load_pretrained_model(model, configs.pretrained_path, configs.gpu_idx, configs.overwrite_global_2_local)
    if logger is not None:
        logger.info('loaded pretrained model at {}'.format(configs.pretrained_path))

# optionally resume from a checkpoint
if configs.resume_path is not None:
    checkpoint = resume_model(configs.resume_path, configs.arch, configs.gpu_idx)
    if hasattr(model, 'module'):
        model.module.load_state_dict(checkpoint['state_dict'])
    else:
        model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    best_val_loss = checkpoint['best_val_loss']
    earlystop_count = checkpoint['earlystop_count']
    configs.start_epoch = checkpoint['epoch'] + 1

if logger is not None:
    logger.info(">>> Loading dataset & getting dataloader...")

2022-12-19 20:56:22,031: logger.py - info(), at Line 38:INFO:
loaded pretrained model at ../checkpoints/ttnet_2nd_phase_opentt/ttnet_2nd_phase_opentt_epoch_30.pth
2022-12-19 20:56:22,032: logger.py - info(), at Line 38:INFO:
>>> Loading dataset & getting dataloader...


##### Datalaoder

In [21]:
train_loader, val_loader, train_sampler = create_train_val_dataloader(configs)
test_loader = create_test_dataloader(configs)

In [22]:
def train_one_epoch(train_loader, model, optimizer, epoch, configs, logger):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')

    progress = ProgressMeter(len(train_loader), [batch_time, data_time, losses],
                             prefix="Train - Epoch: [{}/{}]".format(epoch, configs.num_epochs))

    # switch to train mode
    model.train()
    start_time = time.time()
    for batch_idx, (resized_imgs, org_ball_pos_xy, global_ball_pos_xy, target_events, target_seg) in enumerate(
            tqdm(train_loader)):
        data_time.update(time.time() - start_time)
        batch_size = resized_imgs.size(0)
        target_seg = target_seg.to(configs.device, non_blocking=True)
        resized_imgs = resized_imgs.to(configs.device, non_blocking=True).float()
        pred_ball_global, pred_ball_local, pred_events, pred_seg, local_ball_pos_xy, total_loss, _ = model(
            resized_imgs, org_ball_pos_xy, global_ball_pos_xy, target_events, target_seg)
        # For torch.nn.DataParallel case
        if (not configs.distributed) and (configs.gpu_idx is None):
            total_loss = torch.mean(total_loss)

        # zero the parameter gradients
        optimizer.zero_grad()
        # compute gradient and perform backpropagation
        total_loss.backward()
        optimizer.step()

        if configs.distributed:
            reduced_loss = reduce_tensor(total_loss.data, configs.world_size)
        else:
            reduced_loss = total_loss.data
        losses.update(to_python_float(reduced_loss), batch_size)
        # measure elapsed time
        torch.cuda.synchronize()
        batch_time.update(time.time() - start_time)

        # Log message
        if logger is not None:
            if ((batch_idx + 1) % configs.print_freq) == 0:
                logger.info(progress.get_message(batch_idx))

        start_time = time.time()

    return losses.avg

In [23]:
def evaluate_one_epoch(val_loader, model, epoch, configs, logger):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')

    progress = ProgressMeter(len(val_loader), [batch_time, data_time, losses],
                             prefix="Evaluate - Epoch: [{}/{}]".format(epoch, configs.num_epochs))
    # switch to evaluate mode
    model.eval()
    with torch.no_grad():
        start_time = time.time()
        for batch_idx, (resized_imgs, org_ball_pos_xy, global_ball_pos_xy, target_events, target_seg) in enumerate(
                tqdm(val_loader)):
            data_time.update(time.time() - start_time)
            batch_size = resized_imgs.size(0)
            target_seg = target_seg.to(configs.device, non_blocking=True)
            resized_imgs = resized_imgs.to(configs.device, non_blocking=True).float()
            pred_ball_global, pred_ball_local, pred_events, pred_seg, local_ball_pos_xy, total_loss, _ = model(
                resized_imgs, org_ball_pos_xy, global_ball_pos_xy, target_events, target_seg)

            # For torch.nn.DataParallel case
            if (not configs.distributed) and (configs.gpu_idx is None):
                total_loss = torch.mean(total_loss)

            if configs.distributed:
                reduced_loss = reduce_tensor(total_loss.data, configs.world_size)
            else:
                reduced_loss = total_loss.data
            losses.update(to_python_float(reduced_loss), batch_size)
            # measure elapsed time
            torch.cuda.synchronize()
            batch_time.update(time.time() - start_time)

            # Log message
            if logger is not None:
                if ((batch_idx + 1) % configs.print_freq) == 0:
                    logger.info(progress.get_message(batch_idx))

            start_time = time.time()

    return losses.avg

In [24]:
if logger is not None:
    logger.info('number of batches in train set: {}'.format(len(train_loader)))
    if val_loader is not None:
        logger.info('number of batches in val set: {}'.format(len(val_loader)))
    logger.info('number of batches in test set: {}'.format(len(test_loader)))

if configs.evaluate:
    assert val_loader is not None, "The validation should not be None"
    val_loss = evaluate_one_epoch(val_loader, model, configs.start_epoch - 1, configs, logger)
    print('Evaluate, val_loss: {}'.format(val_loss))


2022-12-19 20:56:22,656: logger.py - info(), at Line 38:INFO:
number of batches in train set: 161
2022-12-19 20:56:22,657: logger.py - info(), at Line 38:INFO:
number of batches in test set: 21


In [25]:
# val_loss = evaluate_one_epoch(test_loader, model, configs.start_epoch - 1, configs, logger)
# val_loss

In [26]:
for epoch in range(configs.start_epoch, configs.num_epochs + 1):
    # Get the current learning rate
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    if logger is not None:
        logger.info('{}'.format('*-' * 40))
        logger.info('{} {}/{} {}'.format('=' * 35, epoch, configs.num_epochs, '=' * 35))
        logger.info('{}'.format('*-' * 40))
        logger.info('>>> Epoch: [{}/{}] learning rate: {:.2e}'.format(epoch, configs.num_epochs, lr))

    if configs.distributed:
        train_sampler.set_epoch(epoch)
    # train for one epoch
    train_loss = train_one_epoch(train_loader, model, optimizer, epoch, configs, logger)
    loss_dict = {'train': train_loss}
    if not configs.no_val:
        val_loss = evaluate_one_epoch(val_loader, model, epoch, configs, logger)
        is_best = val_loss <= best_val_loss
        best_val_loss = min(val_loss, best_val_loss)
        loss_dict['val'] = val_loss

    if not configs.no_test:
        test_loss = evaluate_one_epoch(test_loader, model, epoch, configs, logger)
        loss_dict['test'] = test_loss
    # Write tensorboard
    if tb_writer is not None:
        tb_writer.add_scalars('Loss', loss_dict, epoch)
    # Save checkpoint
    if configs.is_master_node and (is_best or ((epoch % configs.checkpoint_freq) == 0)):
        saved_state = get_saved_state(model, optimizer, lr_scheduler, epoch, configs, best_val_loss,
                                      earlystop_count)
        save_checkpoint(configs.checkpoints_dir, configs.saved_fn, saved_state, is_best, epoch)
    # Check early stop training
    if configs.earlystop_patience is not None:
        earlystop_count = 0 if is_best else (earlystop_count + 1)
        print_string = ' |||\t earlystop_count: {}'.format(earlystop_count)
        if configs.earlystop_patience <= earlystop_count:
            print_string += '\n\t--- Early stopping!!!'
            break
        else:
            print_string += '\n\t--- Continue training..., earlystop_count: {}'.format(earlystop_count)
        if logger is not None:
            logger.info(print_string)
    # Adjust learning rate
    if configs.lr_type == 'plateau':
        assert (not configs.no_val), "Only use plateau when having validation set"
        lr_scheduler.step(val_loss)
    else:
        lr_scheduler.step()



2022-12-19 20:56:24,354: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 20:56:24,355: logger.py - info(), at Line 38:INFO:
=================================== 1/30 ===================================
2022-12-19 20:56:24,356: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 20:56:24,357: logger.py - info(), at Line 38:INFO:
>>> Epoch: [1/30] learning rate: 1.00e-04
100%|██████████| 21/21 [00:12<00:00,  1.74it/s]
2022-12-19 20:57:49,178: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 20:57:49,179: logger.py - info(), at Line 38:INFO:
=================================== 2/30 ===================================
2022-12-19 20:57:49,180: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

save a checkpoint at ../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_2.pth


100%|██████████| 21/21 [00:12<00:00,  1.72it/s]
2022-12-19 21:00:56,974: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:00:56,975: logger.py - info(), at Line 38:INFO:
=================================== 4/30 ===================================
2022-12-19 21:00:56,976: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:00:56,976: logger.py - info(), at Line 38:INFO:
>>> Epoch: [4/30] learning rate: 1.00e-04
100%|██████████| 21/21 [00:11<00:00,  1.77it/s]
2022-12-19 21:02:30,350: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:02:30,351: logger.py - info(), at Line 38:INFO:
=================================== 5/30 ===================================
2022-12-19 21:02:30,352: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

save a checkpoint at ../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_4.pth


100%|██████████| 21/21 [00:11<00:00,  1.80it/s]
2022-12-19 21:04:03,922: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:04:03,923: logger.py - info(), at Line 38:INFO:
=================================== 6/30 ===================================
2022-12-19 21:04:03,924: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:04:03,924: logger.py - info(), at Line 38:INFO:
>>> Epoch: [6/30] learning rate: 1.00e-04
100%|██████████| 21/21 [00:12<00:00,  1.72it/s]
2022-12-19 21:05:38,385: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:05:38,386: logger.py - info(), at Line 38:INFO:
=================================== 7/30 ===================================
2022-12-19 21:05:38,386: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

...

save a checkpoint at ../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_26.pth


100%|██████████| 21/21 [00:11<00:00,  1.76it/s]
2022-12-19 21:38:46,730: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:38:46,731: logger.py - info(), at Line 38:INFO:
=================================== 28/30 ===================================
2022-12-19 21:38:46,732: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:38:46,732: logger.py - info(), at Line 38:INFO:
>>> Epoch: [28/30] learning rate: 4.00e-06
100%|██████████| 21/21 [00:12<00:00,  1.71it/s]
2022-12-19 21:40:22,076: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:40:22,078: logger.py - info(), at Line 38:INFO:
=================================== 29/30 ===================================
2022-12-19 21:40:22,078: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-

save a checkpoint at ../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_28.pth


100%|██████████| 21/21 [00:12<00:00,  1.73it/s]
2022-12-19 21:41:57,138: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:41:57,139: logger.py - info(), at Line 38:INFO:
=================================== 30/30 ===================================
2022-12-19 21:41:57,140: logger.py - info(), at Line 38:INFO:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
2022-12-19 21:41:57,141: logger.py - info(), at Line 38:INFO:
>>> Epoch: [30/30] learning rate: 4.00e-06
100%|██████████| 21/21 [00:12<00:00,  1.71it/s]


save a checkpoint at ../checkpoints/ttnet_3nd_phase/ttnet_3nd_phase_epoch_30.pth


In [27]:
if tb_writer is not None:
    tb_writer.close()
